In [1]:
import re
import glob
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.utils import shuffle

## Train same model as previous notebook

In [3]:
# Load simulated data: load complete training and validation sets.
np.random.seed(11)

# Training set should be one concatenated shuffled list of simulated examples
# Validation set similarly, but also keep information about simulation parameters
# to evaluate performance in specific simulation conditions.
bounds_files = glob.glob("../data/simulData/*_bmBounds.tab")
intensities_files = [re.sub(r"_bmBounds.tab$", "_intensities.tab", bounds_file)
                     for bounds_file in bounds_files]
y, x = shuffle(
    np.concatenate([np.loadtxt(bounds_file) for bounds_file in bounds_files]),
    np.concatenate([np.loadtxt(intensities_file) for intensities_file in intensities_files])
)

In [4]:
def overlap_metric(y_true, y_predicted):
    y_pred_round = tf.round(y_predicted)
    max_start = tf.math.maximum(y_true[:, 0], y_pred_round[:, 0])
    min_end = tf.math.minimum(y_true[:, 1], y_pred_round[:,1])
    overlap = tf.math.maximum(0.0, min_end + 1 - max_start)
    longest_bounds = tf.math.maximum(
        y_true[:, 1] + 1 - y_true[:, 0], 
        y_pred_round[:, 1] + 1 - y_pred_round[:, 0]
        )
    return overlap / longest_bounds

In [ ]:
model = tf.keras.Sequential([
        layers.Dense(40, activation='relu'),
        layers.Dense(2, activation='linear')
    ])
model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
    loss='mse',
    metrics=['mae', overlap_metric]
    )

In [ ]:
history = model.fit(x, y, epochs=1000, batch_size=8192, validation_split=0.33)

Train on 60299 samples, validate on 29701 samples
Epoch 1/1000
60299/60299 [==============================] - 1s 12us/step - loss: 3324.0370 - mean_absolute_error: 43.0560 - overlap_metric: 0.0435 - val_loss: 808.0577 - val_mean_absolute_error: 22.2605 - val_overlap_metric: 0.0749
Epoch 2/1000
60299/60299 [==============================] - 0s 1us/step - loss: 703.8861 - mean_absolute_error: 20.3305 - overlap_metric: 0.0688 - val_loss: 469.8729 - val_mean_absolute_error: 17.4943 - val_overlap_metric: 0.1225
Epoch 3/1000
60299/60299 [==============================] - 0s 1us/step - loss: 329.8925 - mean_absolute_error: 14.1468 - overlap_metric: 0.1104 - val_loss: 224.1466 - val_mean_absolute_error: 11.4307 - val_overlap_metric: 0.1033
Epoch 4/1000
60299/60299 [==============================] - 0s 1us/step - loss: 181.7828 - mean_absolute_error: 10.5280 - overlap_metric: 0.1683 - val_loss: 119.4618 - val_mean_absolute_error: 8.4601 - val_overlap_metric: 0.1824
Epoch 5/1000
60299/60299 [===

Epoch 37/1000
60299/60299 [==============================] - 0s 1us/step - loss: 10.5930 - mean_absolute_error: 2.4964 - overlap_metric: 0.5210 - val_loss: 10.4758 - val_mean_absolute_error: 2.4880 - val_overlap_metric: 0.5232
Epoch 38/1000
60299/60299 [==============================] - 0s 1us/step - loss: 10.3937 - mean_absolute_error: 2.4741 - overlap_metric: 0.5251 - val_loss: 10.3117 - val_mean_absolute_error: 2.4684 - val_overlap_metric: 0.5243
Epoch 39/1000
60299/60299 [==============================] - 0s 1us/step - loss: 10.2227 - mean_absolute_error: 2.4531 - overlap_metric: 0.5292 - val_loss: 10.1181 - val_mean_absolute_error: 2.4452 - val_overlap_metric: 0.5318
Epoch 40/1000
60299/60299 [==============================] - 0s 1us/step - loss: 10.0500 - mean_absolute_error: 2.4323 - overlap_metric: 0.5340 - val_loss: 9.9651 - val_mean_absolute_error: 2.4260 - val_overlap_metric: 0.5323
Epoch 41/1000
60299/60299 [==============================] - 0s 1us/step - loss: 9.9014 - mea

60299/60299 [==============================] - 0s 1us/step - loss: 7.5202 - mean_absolute_error: 2.1065 - overlap_metric: 0.5996 - val_loss: 7.5010 - val_mean_absolute_error: 2.1059 - val_overlap_metric: 0.5957
Epoch 74/1000
60299/60299 [==============================] - 0s 1us/step - loss: 7.4861 - mean_absolute_error: 2.1026 - overlap_metric: 0.6014 - val_loss: 7.4705 - val_mean_absolute_error: 2.1008 - val_overlap_metric: 0.5926
Epoch 75/1000
60299/60299 [==============================] - 0s 1us/step - loss: 7.4453 - mean_absolute_error: 2.0960 - overlap_metric: 0.6016 - val_loss: 7.4565 - val_mean_absolute_error: 2.1002 - val_overlap_metric: 0.6002
Epoch 76/1000
60299/60299 [==============================] - 0s 1us/step - loss: 7.4192 - mean_absolute_error: 2.0930 - overlap_metric: 0.6033 - val_loss: 7.4184 - val_mean_absolute_error: 2.0933 - val_overlap_metric: 0.5938
Epoch 77/1000
60299/60299 [==============================] - 0s 1us/step - loss: 7.3921 - mean_absolute_error: 2.0

60299/60299 [==============================] - 0s 1us/step - loss: 6.6798 - mean_absolute_error: 1.9869 - overlap_metric: 0.6219 - val_loss: 6.7003 - val_mean_absolute_error: 1.9906 - val_overlap_metric: 0.6233
Epoch 110/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.6804 - mean_absolute_error: 1.9871 - overlap_metric: 0.6217 - val_loss: 6.6903 - val_mean_absolute_error: 1.9896 - val_overlap_metric: 0.6210
Epoch 111/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.6660 - mean_absolute_error: 1.9841 - overlap_metric: 0.6220 - val_loss: 6.6944 - val_mean_absolute_error: 1.9904 - val_overlap_metric: 0.6270
Epoch 112/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.6450 - mean_absolute_error: 1.9809 - overlap_metric: 0.6229 - val_loss: 6.6993 - val_mean_absolute_error: 1.9928 - val_overlap_metric: 0.6309
Epoch 113/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.6397 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 6.2206 - mean_absolute_error: 1.9175 - overlap_metric: 0.6334 - val_loss: 6.2272 - val_mean_absolute_error: 1.9137 - val_overlap_metric: 0.6340
Epoch 146/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.1703 - mean_absolute_error: 1.9066 - overlap_metric: 0.6353 - val_loss: 6.1799 - val_mean_absolute_error: 1.9088 - val_overlap_metric: 0.6381
Epoch 147/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.1552 - mean_absolute_error: 1.9062 - overlap_metric: 0.6364 - val_loss: 6.1972 - val_mean_absolute_error: 1.9152 - val_overlap_metric: 0.6319
Epoch 148/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.1626 - mean_absolute_error: 1.9087 - overlap_metric: 0.6348 - val_loss: 6.1409 - val_mean_absolute_error: 1.9042 - val_overlap_metric: 0.6416
Epoch 149/1000
60299/60299 [==============================] - 0s 1us/step - loss: 6.1254 - mean_absolute_error:

...

60299/60299 [==============================] - 0s 1us/step - loss: 2.9172 - mean_absolute_error: 1.2819 - overlap_metric: 0.7426 - val_loss: 3.0125 - val_mean_absolute_error: 1.2987 - val_overlap_metric: 0.7365
Epoch 794/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.0626 - mean_absolute_error: 1.3133 - overlap_metric: 0.7343 - val_loss: 3.0689 - val_mean_absolute_error: 1.3243 - val_overlap_metric: 0.7394
Epoch 795/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.2039 - mean_absolute_error: 1.3542 - overlap_metric: 0.7310 - val_loss: 2.8790 - val_mean_absolute_error: 1.2694 - val_overlap_metric: 0.7476
Epoch 796/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.9440 - mean_absolute_error: 1.2838 - overlap_metric: 0.7407 - val_loss: 3.4775 - val_mean_absolute_error: 1.4081 - val_overlap_metric: 0.7068
Epoch 797/1000
60299/60299 [==============================] - 0s 1us/step - loss: 4.0010 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 2.7817 - mean_absolute_error: 1.2463 - overlap_metric: 0.7492 - val_loss: 3.1724 - val_mean_absolute_error: 1.3633 - val_overlap_metric: 0.7332
Epoch 830/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.2077 - mean_absolute_error: 1.3593 - overlap_metric: 0.7302 - val_loss: 2.8964 - val_mean_absolute_error: 1.2747 - val_overlap_metric: 0.7489
Epoch 831/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.4762 - mean_absolute_error: 1.4225 - overlap_metric: 0.7182 - val_loss: 4.9696 - val_mean_absolute_error: 1.7412 - val_overlap_metric: 0.6292
Epoch 832/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.9662 - mean_absolute_error: 1.5191 - overlap_metric: 0.6937 - val_loss: 3.5817 - val_mean_absolute_error: 1.4337 - val_overlap_metric: 0.7243
Epoch 833/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.0671 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 2.8296 - mean_absolute_error: 1.2584 - overlap_metric: 0.7451 - val_loss: 3.7078 - val_mean_absolute_error: 1.4670 - val_overlap_metric: 0.6987
Epoch 866/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.6241 - mean_absolute_error: 1.4472 - overlap_metric: 0.7046 - val_loss: 2.7510 - val_mean_absolute_error: 1.2394 - val_overlap_metric: 0.7525
Epoch 867/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.9171 - mean_absolute_error: 1.2768 - overlap_metric: 0.7374 - val_loss: 2.7794 - val_mean_absolute_error: 1.2400 - val_overlap_metric: 0.7387
Epoch 868/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.7890 - mean_absolute_error: 1.2483 - overlap_metric: 0.7461 - val_loss: 2.8809 - val_mean_absolute_error: 1.2800 - val_overlap_metric: 0.7462
Epoch 869/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.8704 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 2.6619 - mean_absolute_error: 1.2161 - overlap_metric: 0.7544 - val_loss: 2.6640 - val_mean_absolute_error: 1.2121 - val_overlap_metric: 0.7557
Epoch 902/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.7687 - mean_absolute_error: 1.2460 - overlap_metric: 0.7494 - val_loss: 3.4280 - val_mean_absolute_error: 1.3962 - val_overlap_metric: 0.7067
Epoch 903/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.2956 - mean_absolute_error: 1.3884 - overlap_metric: 0.7239 - val_loss: 2.8991 - val_mean_absolute_error: 1.2785 - val_overlap_metric: 0.7450
Epoch 904/1000
60299/60299 [==============================] - 0s 1us/step - loss: 3.2890 - mean_absolute_error: 1.3943 - overlap_metric: 0.7267 - val_loss: 2.7481 - val_mean_absolute_error: 1.2370 - val_overlap_metric: 0.7556
Epoch 905/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.8331 - mean_absolute_error:

60299/60299 [==============================] - 0s 1us/step - loss: 2.5628 - mean_absolute_error: 1.1906 - overlap_metric: 0.7582 - val_loss: 2.6131 - val_mean_absolute_error: 1.2010 - val_overlap_metric: 0.7607
Epoch 938/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.5940 - mean_absolute_error: 1.1980 - overlap_metric: 0.7555 - val_loss: 2.6172 - val_mean_absolute_error: 1.1988 - val_overlap_metric: 0.7534
Epoch 939/1000
60299/60299 [==============================] - 0s 1us/step - loss: 2.6863 - mean_absolute_error: 1.2244 - overlap_metric: 0.7529 - val_loss: 4.1642 - val_mean_absolute_error: 1.6227 - val_overlap_metric: 0.6982
Epoch 940/1000
60299/60299 [==============================] - 0s 1us/step - loss: 7.4056 - mean_absolute_error: 2.2328 - overlap_metric: 0.5972 - val_loss: 6.8879 - val_mean_absolute_error: 2.1825 - val_overlap_metric: 0.5928
Epoch 941/1000
60299/60299 [==============================] - 0s 1us/step - loss: 4.4297 - mean_absolute_error:

Check same metrics as previous notebook.

In [ ]:
print("Mean absolute error", history.history['val_mean_absolute_error'][-1])
print("Overlap metric", history.history['val_overlap_metric'][-1])

## Validation on different datasets

Simulated datasets, separately

In [ ]:
bounds_files = glob.glob("../data/simulData/*_bmBounds.tab")
intensities_files = [re.sub(r"_bmBounds.tab$", "_intensities.tab", bounds_file)
                     for bounds_file in bounds_files]

In [ ]:
adiposities = []
scales = []
overlaps = []
for bounds_file, intensities_file in zip(bounds_files, intensities_files):
    test_y, test_x = shuffle(np.loadtxt(bounds_file), np.loadtxt(intensities_file))
    values = model.evaluate(test_x, test_y)
    m = re.search(r"/scale(.*)-adip(.*)_bmBounds.tab$", bounds_file)
    scale, adip = [float(x) for x in m.groups()]
    overlap_index = model.metrics_names.index("overlap_metric")
    overlap = values[overlap_index]
    overlaps.append(overlap)
    adiposities.append(adip)
    scales.append(scale)

In [ ]:
ao_data = zip(adiposities, overlaps)
a, o = zip(*sorted(ao_data))

In [ ]:
plt.plot(a, o, 'o')
plt.show()

In [ ]:
so_data = zip(scales, overlaps)
s, o = zip(*sorted(so_data))

In [ ]:
plt.plot(s, o, 'o')
plt.show()

## Real data predictions

In [ ]:
real_data_file = "../data/realData/FS_1334172_layers.csv"
raw = np.loadtxt(real_data_file, delimiter=";")
real_data = np.transpose(raw)
preds = model.predict(real_data)

In [ ]:
np.savetxt("../data/predictions.txt", preds, fmt="%d")